In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

from transformers import TFAutoModel, AutoTokenizer, BertConfig, TFBertModel

In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=25)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=15)
    plt.yticks(tick_marks, classes, fontsize=15)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black", fontsize = 14)

    plt.ylabel('True label', fontsize=20)
    plt.xlabel('Predicted label', fontsize=20)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nir/task_class_2.csv')

print(df.shape)
df.head()

FileNotFoundError: ignored

In [ ]:
df['body'] = df['body'].apply(lambda x: str(x))

In [ ]:
df['len_news_name'] = df['news_name'].apply(lambda x: len(x))
df['len_body'] = df['body'].apply(lambda x: len(x))
df['len_content'] = df['content'].apply(lambda x: len(x))
df.describe()

,Unnamed: 0,match_id,id,after_match,len_news_name,len_body,len_content
count,92997.000000,9.299700e+04,9.299700e+04,92997.000000,92997.000000,92997.000000,92997.000000
mean,46959.977806,6.219752e+05,4.203948e+08,0.764584,71.749422,837.317548,9522.808843
std,27098.972564,3.408401e+05,4.313812e+08,0.424261,20.591112,1044.389169,2722.276930
min,0.000000,1.892500e+04,7.005921e+06,0.000000,19.000000,3.000000,4.000000
25%,23525.000000,2.959910e+05,1.162970e+08,1.000000,57.000000,437.000000,7517.000000
50%,46865.000000,7.631510e+05,1.480367e+08,1.000000,69.000000,634.000000,9188.000000
75%,70394.000000,9.629140e+05,1.024447e+09,1.000000,84.000000,939.000000,11269.000000
max,94106.000000,1.104866e+06,1.040658e+09,1.000000,200.000000,23184.000000,32812.000000


In [ ]:
### UTILITY FUNCTIONS FOR TOKENIZATIONS, MASKS AND SEGMENTS CREATION ###
### from: https://www.kaggle.com/akensert/bert-base-tf2-0-now-huggingface-transformer

def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=True):
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    if double:
    
        input_ids_1, input_masks_1, input_segments_1 = return_id(
            str1, None, 'longest_first', max_sequence_length)

        input_ids_2, input_masks_2, input_segments_2 = return_id(
            str2, None, 'longest_first', max_sequence_length)

        return [input_ids_1, input_masks_1, input_segments_1,
                input_ids_2, input_masks_2, input_segments_2]
    
    else:
        
        input_ids, input_masks, input_segments = return_id(
            str1, str2, 'longest_first', max_sequence_length)

        return [input_ids, input_masks, input_segments,
                None, None, None]        

def compute_input_arrays(df, columns, tokenizer, max_sequence_length, double=True):
    
    input_ids_1, input_masks_1, input_segments_1 = [], [], []
    input_ids_2, input_masks_2, input_segments_2 = [], [], []
    for _, instance in tqdm(df[columns].iterrows(), total=len(df)):
        str1, str2 = instance[columns[0]], instance[columns[1]]

        ids_1, masks_1, segments_1, ids_2, masks_2, segments_2 = \
        convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=double)
        
        input_ids_1.append(ids_1)
        input_masks_1.append(masks_1)
        input_segments_1.append(segments_1)

        input_ids_2.append(ids_2)
        input_masks_2.append(masks_2)
        input_segments_2.append(segments_2)
        
    if double:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32),
                np.asarray(input_ids_2, dtype=np.int32), 
                np.asarray(input_masks_2, dtype=np.int32), 
                np.asarray(input_segments_2, dtype=np.int32)]
    
    else:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32)]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[['content','body']], df['after_match'].values, 
                                                    random_state=42, test_size = 0.2, shuffle=True, stratify=df['after_match'].values)
del df
 
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
### IMPORT TOKENIZER ###

MAX_SEQUENCE_LENGTH = 300

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

In [ ]:
### CREATE SEQUENCES (id, mask, segments) FOR TRAIN AND TEST ###

input_train = compute_input_arrays(X_train, ['content','body'], tokenizer, MAX_SEQUENCE_LENGTH, double=False)
input_test = compute_input_arrays(X_test, ['content','body'], tokenizer, MAX_SEQUENCE_LENGTH, double=False)

  0%|          | 0/74397 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/18600 [00:00<?, ?it/s]

In [ ]:
def simple_bert():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    atn_ = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig()
    config.output_hidden_states = False # Set to True to obtain hidden states
    bert_model = TFBertModel.from_pretrained('sberbank-ai/ruBert-base', from_pt=True)
    
    embedding = bert_model(id_, attention_mask=mask_, token_type_ids=atn_)[0]
    
    x = GlobalAveragePooling1D()(embedding)    
    x = Dropout(0.2)(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inputs=[id_, mask_, atn_], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    
    return model

In [ ]:
model = simple_bert()
model.fit(input_train, y_train, epochs=3, batch_size=6)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Epoch 1/3
12400/12400 [==============================] - 5894s 474ms/step - loss: 0.2534
Epoch 2/3
 8158/12400 [==================>...........] - ETA: 33:33 - loss: 0.2043

In [ ]:
### PREDICT TEST ###

pred_test = np.argmax(model.predict(input_test), axis=1)

In [ ]:
print(classification_report(y_test, pred_test))

In [ ]:
model.save_weights('/content/drive/MyDrive/ColabNotebooks/NIR/data/weights_300_3')

In [ ]:
df = df.head(5000)

In [ ]:
res = pd.DataFrame(np.repeat(df.values, df['len_body'].apply(lambda x: x // 300 + 1).array, axis=0), columns=df.columns).astype(df.dtypes)

In [ ]:
res['numb_start'] = res.groupby(['id']).cumcount()+1

In [ ]:
res

,Unnamed: 0,match_id,team1,team2,broadcast_name,match_time,content,id,news_name,ctime,body,after_match,len_news_name,len_body,len_content,numb_start
0,0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,"Добрый день! Sports.ru поздравляет всех, кто п...",7267034.0,Три тысячи сотрудников милиции будут следить з...,2009-03-13 22:17:53,Более трех тысяч сотрудников столичной милиции...,0,103,994,8142,1
1,0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,"Добрый день! Sports.ru поздравляет всех, кто п...",7267034.0,Три тысячи сотрудников милиции будут следить з...,2009-03-13 22:17:53,Более трех тысяч сотрудников столичной милиции...,0,103,994,8142,2
2,0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,"Добрый день! Sports.ru поздравляет всех, кто п...",7267034.0,Три тысячи сотрудников милиции будут следить з...,2009-03-13 22:17:53,Более трех тысяч сотрудников столичной милиции...,0,103,994,8142,3
3,0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,"Добрый день! Sports.ru поздравляет всех, кто п...",7267034.0,Три тысячи сотрудников милиции будут следить з...,2009-03-13 22:17:53,Более трех тысяч сотрудников столичной милиции...,0,103,994,8142,4
4,1,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,"Добрый день! Sports.ru поздравляет всех, кто п...",7269714.0,Текстовый онлайн матча «Атлетик» – «Реал» – на...,2009-03-14 03:45:43,Сегодня Sports.ru продолжит текстовые трансляц...,0,56,1002,8142,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20392,5204,32419,Аргентина,Южная Корея,перенесенные матчи,2010-06-17 11:30:00,"Здравствуйте, друзья. 2-й тур чемпионата мира-...",71286004.0,Хо Чжон Му: «Сборная Аргентины – это не только...,2010-06-17 12:03:11,Главный тренер сборной Южной Кореи Хо Чжон Му ...,1,53,330,11391,1
20393,5204,32419,Аргентина,Южная Корея,перенесенные матчи,2010-06-17 11:30:00,"Здравствуйте, друзья. 2-й тур чемпионата мира-...",71286004.0,Хо Чжон Му: «Сборная Аргентины – это не только...,2010-06-17 12:03:11,Главный тренер сборной Южной Кореи Хо Чжон Му ...,1,53,330,11391,2
20394,5205,32419,Аргентина,Южная Корея,перенесенные матчи,2010-06-17 11:30:00,"Здравствуйте, друзья. 2-й тур чемпионата мира-...",71289760.0,Самуэль травмировался в матче с Южной Кореей,2010-06-17 15:58:55,Защитник сборной Аргентины Вальтер Самуэль был...,1,44,232,11391,1
20395,5206,32419,Аргентина,Южная Корея,перенесенные матчи,2010-06-17 11:30:00,"Здравствуйте, друзья. 2-й тур чемпионата мира-...",71290795.0,Гутьеррес пропустит матч против сборной Греции,2010-06-17 16:49:30,Полузащитник сборной Аргентины Хонас Гутьеррес...,1,46,420,11391,1


In [ ]:
res['part_content'] = res.apply(lambda row: str(row['content'])[(row['numb_start'] - 1) * 300 : (row['numb_start']) * 300], axis = 1)

In [ ]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20397 entries, 0 to 20396
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      20397 non-null  int64  
 1   match_id        20397 non-null  int64  
 2   team1           20397 non-null  object 
 3   team2           20397 non-null  object 
 4   broadcast_name  20397 non-null  object 
 5   match_time      20397 non-null  object 
 6   content         20397 non-null  object 
 7   id              20397 non-null  float64
 8   news_name       20397 non-null  object 
 9   ctime           20397 non-null  object 
 10  body            20397 non-null  object 
 11  after_match     20397 non-null  int64  
 12  len_news_name   20397 non-null  int64  
 13  len_body        20397 non-null  int64  
 14  len_content     20397 non-null  int64  
 15  numb_start      20397 non-null  int64  
 16  part_content    20397 non-null  object 
dtypes: float64(1), int64(7), object

In [ ]:
res['part_content'] = res['part_content'].apply(lambda x: str(x))
res['news_name'] = res['news_name'].apply(lambda x: str(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(res[['part_content', 'news_name']], res['after_match'].values, 
                                                    random_state=42, test_size = 0.2, shuffle=True, stratify=res['after_match'].values)


 
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(16317, 2) (4080, 2)
(16317,) (4080,)


In [ ]:
### IMPORT TOKENIZER ###

MAX_SEQUENCE_LENGTH_CONTENT = 300
MAX_SEQUENCE_LENGTH_NEWS = 300

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

In [ ]:
### CREATE SEQUENCES (id, mask, segments) FOR TRAIN AND TEST ###

input_train = compute_input_arrays(X_train, ['part_content','news_name'], tokenizer, MAX_SEQUENCE_LENGTH_CONTENT)
input_test = compute_input_arrays(X_test, ['part_content','news_name'], tokenizer, MAX_SEQUENCE_LENGTH_NEWS)

NameError: ignored

In [ ]:
def dual_bert():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    id2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    mask1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    mask2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    atn1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    atn2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    config = BertConfig() 
    config.output_hidden_states = False # Set to True to obtain hidden states
    bert_model1 = TFBertModel.from_pretrained('sberbank-ai/ruBert-base', from_pt=True)
    bert_model2 = TFBertModel.from_pretrained('sberbank-ai/ruBert-base', from_pt=True)
    
    embedding1 = bert_model1(id1, attention_mask=mask1, token_type_ids=atn1)[0]
    embedding2 = bert_model2(id2, attention_mask=mask2, token_type_ids=atn2)[0]
    
    x1 = GlobalAveragePooling1D()(embedding1)
    x2 = GlobalAveragePooling1D()(embedding2)
    
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inputs=[id1, mask1, atn1, id2, mask2, atn2], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    
    return model

In [ ]:
model = dual_bert()
model.fit(input_train, y_train, epochs=3, batch_size=6)

Downloading:   0%|          | 0.00/683M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Epoch 1/3
2720/2720 [==============================] - 2502s 908ms/step - loss: 0.2996
Epoch 2/3
2720/2720 [==============================] - 2469s 908ms/step - loss: 0.0896
Epoch 3/3
2720/2720 [==============================] - 2471s 908ms/step - loss: 0.0585


In [ ]:
### PREDICT TEST ###

pred_test = np.argmax(model.predict(input_test), axis=1) 

In [ ]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97      1425
           1       0.98      0.98      0.98      2655

    accuracy                           0.98      4080
   macro avg       0.97      0.97      0.97      4080
weighted avg       0.98      0.98      0.98      4080



In [ ]:
# два берта 300, 3 эпохи, 8000 строк
model.save_weights('/content/drive/MyDrive/nir/bert2_weights_300_3')

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(res[['part_content', 'news_name', 'match_id', 'id']], res['after_match'].values, 
                                                    random_state=42, test_size = 0.2, shuffle=True, stratify=res['after_match'].values)

In [ ]:
df_res = X_test2

In [ ]:
df_res['predict'] = pred_test

In [ ]:
df_res['after_match'] = y_test

In [ ]:
df_res

,part_content,news_name,match_id,id,predict,after_match
8760,"о конца выполнить установки автора ""камерунско...",Алексей Попов: «Крупные победы поправляют нам ...,19161,8412553.0,1,1
7864,ть сугубо по фактам!Саенко исполнил непонятную...,Премьер-лига. 6-й тур. «Кубань» вырвала победу...,19142,7771803.0,0,1
17949,Приветствуем Всех любителей НХЛ! После долгих ...,Текстовая трансляция матча «Нью-Джерси» – «Тор...,26476,68880965.0,0,0
11029,"в этом, и серий никаких считать ненужно. Ведь...","Сергей Родионов: «Довольны тем, что удалось за...",19208,19411608.0,1,1
11374,му матчу - абсолютная турнирная похожесть сопе...,Алекс: «На месте Вагнера я бы тоже расплакался»,19216,20796605.0,1,1
...,...,...,...,...,...,...
17253,вать. Поехали!За стартовую минуту шайба успела...,"НХЛ. «Детройт» победил «Сан-Хосе», «Бостон» пр...",25838,45030968.0,0,0
18353,зкой дистанции!Герин нанес уже 4 броска по вор...,Илья Ковальчук: «Буллиты – не самая любимая дл...,26626,69556231.0,1,1
10993,"жна будет спасть, хотя душно будет, как ни кру...",Вячеслав Малафеев: «Когда «Зенит» сравнял счет...,19207,19568480.0,1,1
11171,нсляции по техническим причинам.Пока потихонеч...,Дмитрий Хохлов: «Ностальгия по «Локомотиву» не...,19212,20514232.0,0,0


In [ ]:
 df_res = df_res.groupby(['id', 'match_id', 'after_match'], as_index=False)['predict'].sum()

In [ ]:
df_res['predict'] = df_res['predict'].apply(lambda x: 1 if x > 0 else 0)
df_res

,id,match_id,after_match,predict
0,7005921.0,18936,0,0
1,7006514.0,18936,0,0
2,7006598.0,18935,0,0
3,7006881.0,18936,0,0
4,7007361.0,18936,0,0
...,...,...,...,...
2602,71391789.0,32416,1,1
2603,71392916.0,32415,1,1
2604,71392940.0,32415,1,1
2605,71393064.0,32416,1,1


In [ ]:
print(classification_report(df_res['after_match'], df_res['predict']))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       677
           1       0.98      0.99      0.98      1930

    accuracy                           0.98      2607
   macro avg       0.97      0.97      0.97      2607
weighted avg       0.98      0.98      0.98      2607



Итоговая точность получилась 98%

In [ ]:
model = dual_bert()
model.load_weights('/content/drive/MyDrive/ColabNotebooks/NIR/data/bert2_weights_300_3')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/task_class_2.csv')
df = df.tail(5000)
df['body'] = df['body'].apply(lambda x: str(x))
df['len_news_name'] = df['news_name'].apply(lambda x: len(x))
df['len_body'] = df['body'].apply(lambda x: len(x))
df['len_content'] = df['content'].apply(lambda x: len(x))

res = pd.DataFrame(np.repeat(df.values, df['len_body'].apply(lambda x: x // 300 + 1).array, axis=0), columns=df.columns).astype(df.dtypes)
res['numb_start'] = res.groupby(['id']).cumcount()+1
res['part_content'] = res.apply(lambda row: str(row['content'])[(row['numb_start'] - 1) * 300 : (row['numb_start']) * 300], axis = 1)
res['part_content'] = res['part_content'].apply(lambda x: str(x))
res['news_name'] = res['news_name'].apply(lambda x: str(x))


In [ ]:
input_final = compute_input_arrays(res, ['part_content','news_name'], tokenizer, MAX_SEQUENCE_LENGTH_CONTENT)

  0%|          | 0/14484 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
pred_test = np.argmax(model.predict(input_final), axis=1) 

In [ ]:
df_res = res
df_res['predict'] = pred_test
df_res

,Unnamed: 0,match_id,team1,team2,broadcast_name,match_time,content,id,news_name,ctime,body,after_match,len_news_name,len_body,len_content,numb_start,part_content,predict
0,89022,1063895,Йокерит,СКА,КХЛ,2015-10-01 15:30:00,"Привет всем, кто не представляет своей жизни б...",1.033479e+09,"Сергей Гимаев: «Йокерит» проиграл СКА и ЦСКА, ...",2015-10-01 14:34:59,Заслуженный тренер России Сергей Гимаев подели...,0,89,974,11696,1,"Привет всем, кто не представляет своей жизни б...",0
1,89022,1063895,Йокерит,СКА,КХЛ,2015-10-01 15:30:00,"Привет всем, кто не представляет своей жизни б...",1.033479e+09,"Сергей Гимаев: «Йокерит» проиграл СКА и ЦСКА, ...",2015-10-01 14:34:59,Заслуженный тренер России Сергей Гимаев подели...,0,89,974,11696,2,"И было это так...<img class=""broadcast-img"" sr...",0
2,89022,1063895,Йокерит,СКА,КХЛ,2015-10-01 15:30:00,"Привет всем, кто не представляет своей жизни б...",1.033479e+09,"Сергей Гимаев: «Йокерит» проиграл СКА и ЦСКА, ...",2015-10-01 14:34:59,Заслуженный тренер России Сергей Гимаев подели...,0,89,974,11696,3,""" src=""http://s42.radikal.ru/i097/1510/a5/137d...",0
3,89022,1063895,Йокерит,СКА,КХЛ,2015-10-01 15:30:00,"Привет всем, кто не представляет своей жизни б...",1.033479e+09,"Сергей Гимаев: «Йокерит» проиграл СКА и ЦСКА, ...",2015-10-01 14:34:59,Заслуженный тренер России Сергей Гимаев подели...,0,89,974,11696,4,"венных результатов СКА в сезоне, можно уже буд...",0
4,89023,1063895,Йокерит,СКА,КХЛ,2015-10-01 15:30:00,"Привет всем, кто не представляет своей жизни б...",1.033480e+09,Владимир Антипов: «У СКА было меньше дней на в...,2015-10-01 14:41:21,5-кратный чемпион России и серебряный призер ч...,0,103,803,11696,1,"Привет всем, кто не представляет своей жизни б...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14479,94105,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,"Привет всем, кто не представляет своей жизни б...",1.040654e+09,Джастин Браун: «Питтсбург» все наступал и наст...,2016-06-02 19:36:29,Защитник «Сан-Хосе» Джастин Браун оценил ход с...,1,75,387,13483,2,"и этом у многих, смотревших игру, сложилось вп...",1
14480,94106,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,"Привет всем, кто не представляет своей жизни б...",1.040658e+09,Майк Салливан: «Кросби жульничает? Он не делае...,2016-06-02 22:25:14,Главный тренер «Питтсбурга» Майк Салливан отре...,1,98,930,13483,1,"Привет всем, кто не представляет своей жизни б...",1
14481,94106,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,"Привет всем, кто не представляет своей жизни б...",1.040658e+09,Майк Салливан: «Кросби жульничает? Он не делае...,2016-06-02 22:25:14,Главный тренер «Питтсбурга» Майк Салливан отре...,1,98,930,13483,2,"и этом у многих, смотревших игру, сложилось вп...",1
14482,94106,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,"Привет всем, кто не представляет своей жизни б...",1.040658e+09,Майк Салливан: «Кросби жульничает? Он не делае...,2016-06-02 22:25:14,Главный тренер «Питтсбурга» Майк Салливан отре...,1,98,930,13483,3,"х финала, одной скорости может не хватить. Мас...",1


In [ ]:
 df_res = df_res.groupby(['id', 'match_id', 'after_match'], as_index=False)['predict'].sum()
 df_res['predict'] = df_res['predict'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
print(classification_report(df_res['after_match'], df_res['predict']))

              precision    recall  f1-score   support

           0       0.58      0.33      0.42      1443
           1       0.77      0.90      0.83      3557

    accuracy                           0.74      5000
   macro avg       0.67      0.62      0.63      5000
weighted avg       0.71      0.74      0.71      5000



In [ ]:
df.describe()

,Unnamed: 0,match_id,id,after_match,len_news_name,len_body,len_content
count,5000.000000,5.000000e+03,5.000000e+03,5000.000000,5000.000000,5000.000000,5000.000000
mean,91562.098800,1.084411e+06,1.037105e+09,0.711400,77.002400,720.803400,10734.866000
std,1474.984419,1.244672e+04,2.310749e+06,0.453157,20.330319,425.161252,2508.310889
min,89022.000000,1.063895e+06,1.032155e+09,0.000000,20.000000,71.000000,3953.000000
25%,90271.750000,1.069424e+06,1.034811e+09,0.000000,62.000000,410.750000,8910.000000
50%,91562.500000,1.087062e+06,1.037631e+09,1.000000,76.000000,616.000000,10987.000000
75%,92852.250000,1.095554e+06,1.039008e+09,1.000000,89.000000,915.000000,12404.000000
max,94106.000000,1.104866e+06,1.040658e+09,1.000000,163.000000,4116.000000,18849.000000
